# Library Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch, torchvision
from torchvision import transforms
from torch import nn, optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader as DL
from torch.nn.utils import weight_norm as WN
import torch.nn.functional as F

import gc
import os
import cv2
from time import time

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

seed = 42

# Helper Functions

In [2]:
def breaker():
    print("\n" + 50*"-" + "\n")

def head(x, no_of_ele=5):
    print(x[:no_of_ele])
    
def getImages(file_path=None, file_names=None, size=None):
    images = []
    for name in file_names:
        try:
            image = cv2.imread(file_path + name + ".jpg", cv2.IMREAD_GRAYSCALE)
        except AttributeError:
            print(file_path + name)
        if size:
            image = cv2.resize(image, dsize=(size, size), interpolation=cv2.INTER_LANCZOS4)
        images.append(image.reshape(size, size, 1))
    return np.array(images)

# Data Handling

**Loading Image Data**

In [3]:
start_time = time()

ss = pd.read_csv("../input/ranzcr-clip-catheter-line-classification/sample_submission.csv")

ts_img_names = ss["StudyInstanceUID"].values
ts_images = getImages("../input/ranzcr-clip-catheter-line-classification/test/", 
                      ts_img_names, 
                      size=384)

breaker()
print("Time Taken to read data : {:.2f} minutes".format((time() - start_time)/60))
breaker()


--------------------------------------------------

Time Taken to read data : 2.13 minutes

--------------------------------------------------



**Dataset Template**

In [4]:
class DS(Dataset):
    def __init__(this, X=None, y=None, transform=None, mode="train"):
        this.mode = mode
        this.transform = transform
        this.X = X
        if mode == "train":
            this.y = y
                 
    def __len__(this):
        return this.X.shape[0]
    
    def __getitem__(this, idx):
        img = this.transform(this.X[idx])
        if this.mode == "train":
            return img, torch.FloatTensor(this.y[idx])
        else:
            return img

# CNN Configuration and Setup

**Config**

In [5]:
class CFG():
    tr_batch_size = 64
    ts_batch_size = 64
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    in_channels = 1
    OL = 11
    
    def __init__(this, filter_sizes=[64, 128, 256, 512], HL=[2048], AP_size=3, epochs=50, n_folds=5):
        this.filter_sizes = filter_sizes
        this.HL = HL
        this.AP_size = AP_size
        this.epochs = epochs
        this.n_folds = n_folds

**Setup**

In [6]:
class CNN(nn.Module):
    def __init__(this, in_channels=1, filter_sizes=None, HL=None, OL=None, AP_size=3, DP=0.5):

        super(CNN, this).__init__()

        this.AP_ = nn.AdaptiveAvgPool2d(output_size=AP_size)
        this.MP_ = nn.MaxPool2d(kernel_size=2)
        this.DP_ = nn.Dropout(p=0.5)
        
        this.CN1_1 = nn.Conv2d(in_channels=in_channels, out_channels=filter_sizes[0], kernel_size=3, stride=1, padding=1)
        this.BN1_1 = nn.BatchNorm2d(num_features=filter_sizes[0], eps=1e-5)
        this.CN1_2 = nn.Conv2d(in_channels=filter_sizes[0], out_channels=filter_sizes[0], kernel_size=3, stride=1, padding=1)
        this.BN1_2 = nn.BatchNorm2d(num_features=filter_sizes[0], eps=1e-5)

        this.CN2_1 = nn.Conv2d(in_channels=filter_sizes[0], out_channels=filter_sizes[1], kernel_size=3, stride=1, padding=1)
        this.BN2_1 = nn.BatchNorm2d(num_features=filter_sizes[1], eps=1e-5)
        this.CN2_2 = nn.Conv2d(in_channels=filter_sizes[1], out_channels=filter_sizes[1], kernel_size=3, stride=1, padding=1)
        this.BN2_2 = nn.BatchNorm2d(num_features=filter_sizes[1], eps=1e-5)
        
        this.CN3_1 = nn.Conv2d(in_channels=filter_sizes[1], out_channels=filter_sizes[2], kernel_size=3, stride=1, padding=1)
        this.BN3_1 = nn.BatchNorm2d(num_features=filter_sizes[2], eps=1e-5)
        this.CN3_2 = nn.Conv2d(in_channels=filter_sizes[2], out_channels=filter_sizes[2], kernel_size=3, stride=1, padding=1)
        this.BN3_2 = nn.BatchNorm2d(num_features=filter_sizes[2], eps=1e-5)

        this.CN4_1 = nn.Conv2d(in_channels=filter_sizes[2], out_channels=filter_sizes[3], kernel_size=3, stride=1, padding=1)
        this.BN4_1 = nn.BatchNorm2d(num_features=filter_sizes[3], eps=1e-5)
        this.CN4_2 = nn.Conv2d(in_channels=filter_sizes[3], out_channels=filter_sizes[3], kernel_size=3, stride=1, padding=1)
        this.BN4_2 = nn.BatchNorm2d(num_features=filter_sizes[3], eps=1e-5)

        this.FC1 = nn.Linear(in_features=filter_sizes[3]*AP_size*AP_size, out_features=HL[0])
        this.FC2 = nn.Linear(in_features=HL[0], out_features=OL)
        
    def getOptimizer(this, lr=1e-3, wd=0):
        return optim.Adam(this.parameters(), lr=lr, weight_decay=wd)

    def getPlateauLR(this, optimizer=None, patience=5, eps=1e-8):
        return optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, patience=patience, eps=eps, verbose=True)

    def forward(this, x):
        x = F.relu(this.MP_(this.BN1_2(this.CN1_2(this.BN1_1(this.CN1_1(x))))))
        x = F.relu(this.MP_(this.BN2_2(this.CN2_2(this.BN2_1(this.CN2_1(x))))))
        x = F.relu(this.MP_(this.BN3_2(this.CN3_2(this.BN3_1(this.CN3_1(x))))))
        x = F.relu(this.MP_(this.BN4_2(this.CN4_2(this.BN4_1(this.CN4_1(x))))))

        x = this.AP_(x)
        x = x.view(x.shape[0], -1)

        x = F.relu(this.DP_(this.FC1(x)))
        x = this.FC2(x)
        
        return x

**Predict Function**

In [7]:
def predict_(model=None, dataloader=None, device=None, path=None):
    if path:
        model.load_state_dict(torch.load(path)["model_state_dict"])

    model.to(device)
    model.eval()

    y_pred = torch.zeros(1, 11).to(device)

    for X in dataloader:
        X = X.to(device)
        with torch.no_grad():
            Pred = torch.sigmoid(model(X))
        y_pred = torch.cat((y_pred, Pred), dim=0)
    
    return y_pred[1:].detach().cpu().numpy()

# Submission 

In [8]:
cfg = CFG(filter_sizes=[64, 128, 256, 512], HL=[2048], epochs=None, n_folds=None)

transform = transforms.Compose([transforms.ToTensor(), ])

ts_data_setup = DS(X=ts_images, y=None, transform=transform, mode="test")
ts_data = DL(ts_data_setup, batch_size=cfg.ts_batch_size, shuffle=False)

model = CNN(filter_sizes=cfg.filter_sizes, HL=cfg.HL, OL=cfg.OL, AP_size=cfg.AP_size)

y_pred = predict_(model=model, dataloader=ts_data, device=cfg.device, path="../input/rccl-384-10l-train/Epoch_24.pt")
y_pred = np.clip(y_pred, 1e-15, 1-1e-15)

ss.iloc[:, 1:] = y_pred
ss.to_csv("./submission.csv", index=False)
ss.head(5)

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present
0,1.2.826.0.1.3680043.8.498.46923145579096002617...,0.008160,0.241234,7.427457e-01,0.013000,0.027711,1.506694e-01,0.752132,0.100817,0.303184,0.933354,9.503056e-01
1,1.2.826.0.1.3680043.8.498.84006870182611080091...,0.000001,0.000002,1.397642e-07,0.000036,0.000008,6.454061e-07,0.000001,0.047738,0.101744,0.868951,2.820443e-07
2,1.2.826.0.1.3680043.8.498.12219033294413119947...,0.000039,0.000160,8.165937e-05,0.001054,0.000782,3.845184e-04,0.000631,0.057162,0.270184,0.734273,3.596837e-05
3,1.2.826.0.1.3680043.8.498.84994474380235968109...,0.004434,0.203048,7.407291e-01,0.039214,0.009442,8.331118e-01,0.120162,0.065231,0.258453,0.884859,3.545037e-01
4,1.2.826.0.1.3680043.8.498.35798987793805669662...,0.000117,0.000683,6.709865e-04,0.001893,0.003409,8.556456e-05,0.008507,0.106428,0.294934,0.598096,6.776825e-05
